# Import haplotype data to sgkit

Convert MalariaGEN data from scikit-allel VCF Zarr format, to sgkit Zarr format. This uses the `vcfzarr_to_zarr` function that has been optimized to avoid high-memory usage. (See https://github.com/pystatgen/sgkit/pull/324 and the linked issues for details.)

We also use the [rechunker](https://rechunker.readthedocs.io/en/latest/) library to rechunk to the desired chunk sizes.

In [1]:
%run setup.ipynb

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from dask.diagnostics import ProgressBar
from sgkit.io.vcfzarr_reader import vcfzarr_to_zarr

In [4]:
input = here() / 'data/external/ag1000g/phase2/AR1/haplotypes/main/zarr/ag1000g.phase2.ar1.haplotypes/'
output = here() / 'data/sgkit/ag1000g_import_haplotypes.zarr'
contigs = ["2R", "2L", "3R", "3L"] # note R is before L; skip X since it has fewer samples (1099 vs 1164)

In [5]:
with ProgressBar():
    vcfzarr_to_zarr(input, output, contigs=contigs, grouped_by_contig=True, consolidated=True)

[########################################] | 100% Completed |  2.9s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 37.1s
[########################################] | 100% Completed |  2.2s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 28.3s
[########################################] | 100% Completed |  2.8s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 37.0s
[########################################] | 100% Completed |  2.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 27.0s
[########################################] | 100% Completed |  2min 26.6s


Have a look at the dataset that's been created, by reading it with Xarray.

In [6]:
import xarray as xr

In [7]:
ds = xr.open_zarr(str(output), concat_characters=False)
ds

,Array,Chunk
Bytes,92.20 GB,62.91 MB
Shape,"(39604636, 1164, 2)","(524288, 60, 2)"
Count,1521 Tasks,1520 Chunks
Type,int8,numpy.ndarray
,Array,Chunk
Bytes,92.20 GB,62.91 MB
Shape,"(39604636, 1164, 2)","(524288, 60, 2)"
Count,1521 Tasks,1520 Chunks
Type,bool,numpy.ndarray
,Array,Chunk


The dataset is chunked, in the `samples` dimension (chunk size is `60`, total number of samples is `1164`), but the popgen functions don't support chunking in that dimension, so we need to rechunk to have a single chunk in that dimension. We can do that using [rechunker](https://rechunker.readthedocs.io/en/latest/) working directly on Zarr groups.

In [8]:
source_group = zarr.open(str(output))
target_chunks = {"call_genotype": (524288, 1164, 2), "call_genotype_mask": (524288, 1164, 2), "sample_id": None, "variant_allele": None, "variant_contig": None, "variant_position": None}
max_mem = '2GB'

target_store = str(here() / 'data/sgkit/ag1000g_haplotypes.zarr')
temp_store = str(here() / 'data/sgkit/ag1000g_haplotypes_rechunked_tmp.zarr')

In [10]:
from rechunker import api as rechunker_api
plan = rechunker_api.rechunk(source_group, target_chunks, max_mem, target_store)

In [11]:
with ProgressBar():
    plan.execute()

[########################################] | 100% Completed |  3min 17.0s


Now when we look at the dataset it is chunked as we want it.

In [12]:
ds = xr.open_zarr(target_store, concat_characters=False)
ds

<xarray.Dataset>
Dimensions:             (alleles: 2, ploidy: 2, samples: 1164, variants: 39604636)
Dimensions without coordinates: alleles, ploidy, samples, variants
Data variables:
    call_genotype       (variants, samples, ploidy) int8 dask.array<chunksize=(524288, 1164, 2), meta=np.ndarray>
    call_genotype_mask  (variants, samples, ploidy) bool dask.array<chunksize=(524288, 1164, 2), meta=np.ndarray>
    sample_id           (samples) <U8 dask.array<chunksize=(1164,), meta=np.ndarray>
    variant_allele      (variants, alleles) |S1 dask.array<chunksize=(4194304, 2), meta=np.ndarray>
    variant_contig      (variants) int32 dask.array<chunksize=(4194304,), meta=np.ndarray>
    variant_position    (variants) int32 dask.array<chunksize=(4194304,), meta=np.ndarray>
Attributes:
    contigs:                    ['2R', '2L', '3R', '3L']
    max_variant_allele_length:  1